In [ ]:
import torch
from torch.autograd import Variable
import numpy as np
from collections import defaultdict
import matplotlib.pyplot as plt
import cv2
import pickle
from random import *
import gc
from torch.distributions.multivariate_normal import MultivariateNormal
import torch.optim as optim
from time import gmtime, strftime
import os
from PIL import Image
%matplotlib inline

In [ ]:
n_desc_per_img = 10

## Read Data on System

In [ ]:
def read_input(dataset='CUB', process='resize'):
    # Define File Destinations
    test_desc_fname = dataset+'/desc/test/char-CNN-RNN-embeddings.npy'
    train_desc_fname = dataset+'/desc/train/char-CNN-RNN-embeddings.npy'
    
    test_files_fname = dataset+'/desc/test/filenames.pickle'
    train_files_fname = dataset+'/desc/train/filenames.pickle'
    text_dir = dataset+'/desc/text_c10/'
    
    test_img_dir = dataset+'/images/'
    train_img_dir = dataset+'/images/'
    
    train_s1_imgs = []
    test_s1_imgs = []
    train_s2_imgs = []
    test_s2_imgs = []
    train_s1_emds = []
    test_s1_emds = []
    train_s2_emds = []
    test_s2_emds = []
    train_text = []
    test_text = []
    
    #Load Training Data
    print('Loading Training Data...')
    train_embed = np.load(train_desc_fname)
    train_embed_shape = train_embed.shape
    
    with open(train_files_fname,'rb') as file:
        dat = pickle.load(file)
    for i in range(len(dat)):
        for l in open(text_dir+dat[i]+'.txt','rb'):
            train_text.append(l.strip().decode('utf-8'))
        
        img = cv2.imread(train_img_dir+dat[i]+'.jpg',1)
        if process == 'resize':
            img_s1 = cv2.resize(img,(64,64),interpolation=cv2.INTER_AREA)
            img_s2 = cv2.resize(img,(256,256),interpolation=cv2.INTER_AREA)
        else:
            pass
        img_s1 = np.transpose(img_s1,(2,0,1))
        img_s2 = np.transpose(img_s2,(2,0,1))
        train_s1_imgs.append(img_s1)
        train_s2_imgs.append(img_s2)
        for j in range(train_embed_shape[1]):
#             neg_img = randint(0,train_embed_shape[0]-1)
#             while neg_img == i:
#                 neg_img = randint(0,train_embed_shape[0]-1)
#             neg_idx = randint(0,train_embed_shape[1]-1)
#             train_s1_imgs.append(img_s1)
            train_s1_emds.append(train_embed[i,j,:])
#             train_s1_imgs.append(img_s1)
#             train_s1_emds.append(train_embed[neg_img,neg_idx,:])
#             train_s2_imgs.append(img_s2)
            train_s2_emds.append(train_embed[i,j,:])
#             train_s2_imgs.append(img_s2)
#             train_s2_emds.append(train_embed[neg_img,neg_idx,:])
#             train_s1_data.append((img_s1,train_embed[i,j,:],1))
#             train_s1_data.append((img_s1,train_embed[neg_img,neg_idx,:],0))
#             train_s2_data.append((img_s2,train_embed[i,j,:],1))
#             train_s2_data.append((img_s2,train_embed[neg_img,neg_idx,:],0))
        
        if i%1000 == 0:
            print(i)
        
    #Load Testing Data
    print('Loading Testing Data...')
    test_embed = np.load(test_desc_fname)
    test_embed_shape = test_embed.shape
    
    with open(test_files_fname,'rb') as file:
        dat = pickle.load(file)
    for i in range(len(dat)):
        for l in open(text_dir+dat[i]+'.txt','rb'):
            test_text.append(l.strip().decode('utf-8'))
        
        img = cv2.imread(test_img_dir+dat[i]+'.jpg',1)
        if process == 'resize':
            img_s1 = cv2.resize(img,(64,64),interpolation=cv2.INTER_AREA)
            img_s2 = cv2.resize(img,(256,256),interpolation=cv2.INTER_AREA)
        else:
            pass
        img_s1 = np.transpose(img_s1,(2,0,1))
        img_s2 = np.transpose(img_s2,(2,0,1))
        test_s1_imgs.append(img_s1)
        test_s2_imgs.append(img_s2)
        for j in range(test_embed_shape[1]):
#             neg_img = randint(0,test_embed_shape[0]-1)
#             while neg_img == i:
#                 neg_img = randint(0,test_embed_shape[0]-1)
#             neg_idx = randint(0,test_embed_shape[1]-1)
#             test_s1_imgs.append(img_s1)
            test_s1_emds.append(train_embed[i,j,:])
#             test_s1_imgs.append(img_s1)
#             test_s1_emds.append(train_embed[neg_img,neg_idx,:])
#             test_s2_imgs.append(img_s2)
            test_s2_emds.append(train_embed[i,j,:])
#             test_s2_imgs.append(img_s2)
#             test_s2_emds.append(train_embed[neg_img,neg_idx,:])
#             test_s1_data.append((img_s1,test_embed[i,j,:],1))
#             test_s1_data.append((img_s1,test_embed[neg_img,neg_idx,:],0))
#             test_s2_data.append((img_s2,test_embed[i,j,:],1))
#             test_s2_data.append((img_s2,test_embed[neg_img,neg_idx,:],0))
            
        if i%1000 == 0:
            print(i)
    return train_s1_imgs, train_s1_emds, test_s1_imgs, test_s1_emds, train_s2_imgs, train_s2_emds, test_s2_imgs, test_s2_emds, train_text, test_text
#     return train_s1_data, test_s1_data, train_s2_data, test_s2_data, train_text, test_text

## StackGAN Implementation

### Loss Functions

In [ ]:
loss_criterion = torch.nn.BCELoss()

def KL_Loss(mu, sigma):
    kl_ele = mu.pow(2).add_(sigma.pow(2)).mul_(-1).add_(1).add_(sigma.pow(2).log_())
    kl = torch.mean(kl_ele).mul_(-0.5)
    return kl

def loss_gen(disc_fake, mu, sigma, kl_lambda, batch_size):
    g_fake = -loss_criterion(disc_fake, Variable(torch.FloatTensor([0]*batch_size), requires_grad=False).cuda())
    kl_loss = KL_Loss(mu, sigma)
    g_total = g_fake + (kl_lambda*kl_loss)
    return g_total
    
def loss_disc(disc_real, disc_fake, disc_wrong, batch_size):
    d_real = loss_criterion(disc_real, Variable(torch.FloatTensor([1]*batch_size), requires_grad=False).cuda())
    d_fake = loss_criterion(disc_fake, Variable(torch.FloatTensor([0]*batch_size), requires_grad=False).cuda())
    d_wrong = loss_criterion(disc_wrong, Variable(torch.FloatTensor([0]*batch_size), requires_grad=False).cuda())
    d_total = d_real + ((d_fake + d_wrong)*0.5)
    return d_total

### Model

In [ ]:
def upSample(c_in, c_out):
    mod = torch.nn.Sequential(
        torch.nn.Upsample(scale_factor=2, mode='nearest'),
        torch.nn.Conv2d(c_in, c_out, kernel_size = 3, stride = 1, padding = 1, bias = False),
        torch.nn.BatchNorm2d(c_out),
        torch.nn.ReLU(inplace=True))
    return mod

In [ ]:
class cond_aug(torch.nn.Module):
    def __init__(self, embedding_dim=1024, cond_dim=128):
        super(cond_aug,self).__init__()
        self.embedding_dim = embedding_dim
        self.cond_dim = cond_dim
        
        self.fc_mu = torch.nn.Linear(self.embedding_dim, self.cond_dim)
        self.fc_sigma = torch.nn.Linear(self.embedding_dim, self.cond_dim)
        
        torch.nn.init.xavier_normal_(self.fc_mu.weight)
        torch.nn.init.xavier_normal_(self.fc_sigma.weight)
        self.relu = torch.nn.ReLU(inplace=True)
        
    def forward(self, x, batch_size=64):
        mu = self.relu(self.fc_mu(x))
        logvar = self.relu(self.fc_sigma(x))
        sigma = logvar.mul(0.5).exp_()
        dist = MultivariateNormal(torch.zeros(self.cond_dim).cuda(), torch.eye(self.cond_dim).cuda())
        eps = Variable(dist.sample()).cuda().view(1,-1).repeat(batch_size,1)
        
        c = mu + (sigma * eps)
        return mu, sigma, c

In [ ]:
class ResBlock(torch.nn.Module):
    def __init__(self, channel_num):
        super(ResBlock, self).__init__()
        self.block = torch.nn.Sequential(
            torch.nn.Conv2d(channel_num, channel_num, kernel_size = 3, stride = 1, padding = 1, bias = False),
            torch.nn.BatchNorm2d(channel_num),
            torch.nn.ReLU(True),
            torch.nn.Conv2d(channel_num, channel_num, kernel_size = 3, stride = 1, padding = 1, bias = False),
            torch.nn.BatchNorm2d(channel_num))
        self.relu = torch.nn.ReLU(inplace=True)
        

    def forward(self, x):
        residual = x
        out = self.block(x)
        out += residual
        out = self.relu(out)
        return out

In [ ]:
class stage1_gen(torch.nn.Module):
    def __init__(self, embedding_dim=1024, cond_dim=128, noise_dim=100, ups_input_dim=1024):
        super(stage1_gen,self).__init__()
        self.embedding_dim = embedding_dim
        self.cond_dim = cond_dim
        self.noise_dim = noise_dim
        self.ups_input_dim = ups_input_dim
        self.conc_dim = self.cond_dim + self.noise_dim
        
        self.dist = MultivariateNormal(torch.zeros(self.noise_dim).cuda(), torch.eye(self.noise_dim).cuda())
        self.augm = cond_aug(self.embedding_dim, self.cond_dim)
        self.ups_input = torch.nn.Sequential(
                    torch.nn.Linear(self.conc_dim, self.ups_input_dim*4*4, bias=False),
                    torch.nn.BatchNorm1d(self.ups_input_dim*4*4),
                    torch.nn.ReLU(inplace=True))
        self.upsample1 = upSample(self.ups_input_dim,self.ups_input_dim//2)     
        self.upsample2 = upSample(self.ups_input_dim//2,self.ups_input_dim//4)
        self.upsample3 = upSample(self.ups_input_dim//4,self.ups_input_dim//8)
        self.upsample4 = upSample(self.ups_input_dim//8,self.ups_input_dim//16)
        self.gen_img = torch.nn.Sequential(
                torch.nn.Conv2d(self.ups_input_dim//16, 3, kernel_size = 3, stride = 1, padding = 1, bias = False),
                torch.nn.Tanh())
        
    def forward(self, x, batch_size=64):
        z = Variable(self.dist.sample()).cuda().view(1,-1).repeat(batch_size,1)
        mu, sigma, c = self.augm(x, batch_size=batch_size)
#         c = c.view(1,-1)
        inp = torch.cat((c,z),1)
        
        x = self.ups_input(inp)
        x = x.view(-1,self.ups_input_dim,4,4)
        x = self.upsample1(x)
        x = self.upsample2(x)
        x = self.upsample3(x)
        x = self.upsample4(x)
        fake_img = self.gen_img(x)
        
        return fake_img, mu, sigma

In [ ]:
class stage1_disc(torch.nn.Module):
    def __init__(self, cond_dim=128, down_dim=64):
        super(stage1_disc,self).__init__()
        self.cond_dim = cond_dim
        self.down_dim = down_dim
        
        self.enc_img = torch.nn.Sequential(
            torch.nn.Conv2d(3, self.down_dim, kernel_size = 4, stride = 2, padding = 1, bias = False),
            torch.nn.LeakyReLU(0.2, inplace=True),
            torch.nn.Conv2d(self.down_dim, self.down_dim*2, kernel_size = 4, stride = 2, padding = 1, bias = False),
            torch.nn.BatchNorm2d(self.down_dim*2),
            torch.nn.LeakyReLU(0.2, inplace=True),
            torch.nn.Conv2d(self.down_dim*2, self.down_dim*4, kernel_size = 4, stride = 2, padding = 1, bias = False),
            torch.nn.BatchNorm2d(self.down_dim*4),
            torch.nn.LeakyReLU(0.2, inplace=True),
            torch.nn.Conv2d(self.down_dim*4, self.down_dim*8, kernel_size = 4, stride = 2, padding = 1, bias = False),
            torch.nn.BatchNorm2d(self.down_dim*8),
            torch.nn.LeakyReLU(0.2, inplace=True))
        
        self.get_logits = torch.nn.Sequential(
            torch.nn.Conv2d(self.down_dim*8+self.cond_dim, self.down_dim*8, kernel_size = 3, stride = 1, padding = 1, bias = False),
            torch.nn.BatchNorm2d(self.down_dim*8),
            torch.nn.LeakyReLU(0.2, inplace=True),
            torch.nn.Conv2d(self.down_dim*8, 1, kernel_size = 4, stride = 4),
            torch.nn.Sigmoid())
        
    def forward(self, image, cond_vec):
        x = self.enc_img(image)
        y = cond_vec.view(-1, self.cond_dim, 1, 1)
        y = y.repeat(1,1,4,4)
        z = torch.cat((x,y),1) # N x (128 + 512) x 4 x 4
        out = self.get_logits(z).view(-1)
        
        return out

In [ ]:
class stage2_gen(torch.nn.Module):
    def __init__(self, down_dim=128, embedding_dim=1024, cond_dim=128, num_residuals=4):
        super(stage2_gen,self).__init__()
        self.down_dim = down_dim
        self.embedding_dim = embedding_dim
        self.cond_dim = cond_dim
        
        self.downsampler = torch.nn.Sequential(
            torch.nn.Conv2d(3, self.down_dim, kernel_size = 3, stride = 1, padding = 1, bias = False),
            torch.nn.ReLU(inplace=True),
            torch.nn.Conv2d(self.down_dim, self.down_dim * 2, kernel_size = 4, stride = 2, padding = 1, bias=False),
            torch.nn.BatchNorm2d(self.down_dim * 2),
            torch.nn.ReLU(inplace=True),
            torch.nn.Conv2d(self.down_dim * 2, self.down_dim * 4, kernel_size = 4, stride = 2, padding = 1, bias=False),
            torch.nn.BatchNorm2d(self.down_dim * 4),
            torch.nn.ReLU(inplace=True))
        
        self.augm = cond_aug(self.embedding_dim, self.cond_dim)
        
        self.joint_proc = torch.nn.Sequential(
            torch.nn.Conv2d(self.down_dim * 4 + self.cond_dim, self.down_dim * 4, kernel_size = 3, stride = 1, padding = 1, bias = False),
            torch.nn.BatchNorm2d(self.down_dim * 4),
            torch.nn.ReLU(True))
        
        self.layers = []
        for i in range(num_residuals):
            self.layers.append(ResBlock(self.down_dim * 4))
        self.residual = torch.nn.Sequential(*self.layers)
        
        self.upsample1 = upSample(self.down_dim * 4, self.down_dim * 2)
        self.upsample2 = upSample(self.down_dim * 2, self.down_dim)
        self.upsample3 = upSample(self.down_dim, self.down_dim // 2)
        self.upsample4 = upSample(self.down_dim // 2, self.down_dim // 4)
        self.gen_img = torch.nn.Sequential(
            torch.nn.Conv2d(self.down_dim // 4, 3, kernel_size = 3, stride = 1, padding = 1, bias = False),
            torch.nn.Tanh())

        
        
    def forward(self, text_embedding, stage1_image, batch_size=64):
        encoded_img = self.downsampler(stage1_image) # --> N x 512 x 16 x 16
        mu, sigma, c = self.augm(text_embedding, batch_size=batch_size)
        c = c.view(-1, self.cond_dim, 1, 1) # --> N x 128 x 1 x 1
        c = c.repeat(1, 1, 16, 16) # --> N x 128 x 16 x 16
        conc_inp = torch.cat([encoded_img, c], 1) # --> N x 640 x 16 x 16
        
        conc_out = self.joint_proc(conc_inp) # --> N x 512 x 16 x 16
        conc_out = self.residual(conc_out)   # --> N x 512 x 16 x 16
        
        conc_out = self.upsample1(conc_out) # --> N x 256 x 32 x 32
        conc_out = self.upsample2(conc_out) # --> N x 128 x 64 x 64
        conc_out = self.upsample3(conc_out) # --> N x 64 x 128 x 128
        conc_out = self.upsample4(conc_out) # --> N x 32 x 256 x 256

        fake_img = self.gen_img(conc_out) # --> N x 3 x 256 x 256
        return fake_img, mu, sigma

In [ ]:
class stage2_disc(torch.nn.Module):
    def __init__(self, cond_dim=128, down_dim=64):
        super(stage2_disc,self).__init__()
        self.cond_dim = cond_dim
        self.down_dim = down_dim
        
        self.enc_img = torch.nn.Sequential(
            torch.nn.Conv2d(3, self.down_dim, kernel_size = 4, stride = 2, padding = 1, bias = False),
            torch.nn.LeakyReLU(0.2, inplace=True),
            torch.nn.Conv2d(self.down_dim, self.down_dim*2, kernel_size = 4, stride = 2, padding = 1, bias = False),
            torch.nn.BatchNorm2d(self.down_dim*2),
            torch.nn.LeakyReLU(0.2, inplace=True),
            torch.nn.Conv2d(self.down_dim*2, self.down_dim*4, kernel_size = 4, stride = 2, padding = 1, bias = False),
            torch.nn.BatchNorm2d(self.down_dim*4),
            torch.nn.LeakyReLU(0.2, inplace=True),
            torch.nn.Conv2d(self.down_dim*4, self.down_dim*8, kernel_size = 4, stride = 2, padding = 1, bias = False),
            torch.nn.BatchNorm2d(self.down_dim*8),
            torch.nn.LeakyReLU(0.2, inplace=True),
            torch.nn.Conv2d(self.down_dim*8, self.down_dim*16, kernel_size = 4, stride = 2, padding = 1, bias = False),
            torch.nn.BatchNorm2d(self.down_dim*16),
            torch.nn.LeakyReLU(0.2, inplace=True),
            torch.nn.Conv2d(self.down_dim*16, self.down_dim*32, kernel_size = 4, stride = 2, padding = 1, bias = False),
            torch.nn.BatchNorm2d(self.down_dim*32),
            torch.nn.LeakyReLU(0.2, inplace=True),
            torch.nn.Conv2d(self.down_dim*32, self.down_dim*16, kernel_size = 3, stride = 1, padding = 1, bias = False),
            torch.nn.BatchNorm2d(self.down_dim * 16),
            torch.nn.LeakyReLU(0.2, inplace=True),
            torch.nn.Conv2d(self.down_dim*16, self.down_dim*8, kernel_size = 3, stride = 1, padding = 1, bias = False),
            torch.nn.BatchNorm2d(self.down_dim * 8),
            torch.nn.LeakyReLU(0.2, inplace=True))   # 4 * 4 * ndf * 8)
        
        self.get_logits = torch.nn.Sequential(
            torch.nn.Conv2d(self.down_dim*8+self.cond_dim, self.down_dim*8, kernel_size = 3, stride = 1, padding = 1, bias = False),
            torch.nn.BatchNorm2d(self.down_dim*8),
            torch.nn.LeakyReLU(0.2, inplace=True),
            torch.nn.Conv2d(self.down_dim*8, 1, kernel_size = 4, stride = 4),
            torch.nn.Sigmoid())
        
    def forward(self, image, cond_vec):
        x = self.enc_img(image)
        y = cond_vec.view(-1, self.cond_dim, 1, 1)
        y = y.repeat(1,1,4,4)
#         print(x.shape)
#         print(y.shape)
        z = torch.cat((x,y),1) # N x (128 + 512) x 4 x 4
        out = self.get_logits(z).view(-1)
        
        return out

In [ ]:
def train_s1(gen, disc, train_s1_imgs, train_s1_emds, batch_size=64, epochs=10, eta=0.0001, opt=optim.Adam,
             model_name='A_Model_Has_No_Name', kl_lambda=2.0, embedding_dim=1024, cond_dim=128):
    gen.cuda()
    disc.cuda()
    
    optim_gen = opt(gen.parameters(), lr=eta)
    optim_disc = opt(disc.parameters(), lr=eta)
    
    print('Training Stage 1...')
    
    l_tr_g = []
    l_tr_d = []
    iter_tr = []
    
#     n_train = len(train_s1_imgs)
    desc_shape = train_s1_emds[0].shape
    img_shape = train_s1_imgs[0].shape
    
    file_path = "./observations/"
    directory = os.path.dirname(file_path)
    if not os.path.exists(directory):
        os.makedirs(directory)
    file_path = "./saved_models/"
    directory = os.path.dirname(file_path)
    if not os.path.exists(directory):
        os.makedirs(directory)
    ts = strftime("%Y-%m-%d__%Hh%Mm%Ss_S1_" + model_name, gmtime())
    
    real_pairs = []
    for i in range(len(train_s1_imgs)):
        for j in range(n_desc_per_img):
            real_pairs.append((i,i * n_desc_per_img + j))
    
    n_train = len(real_pairs)
    print('Training Size = ',n_train)
    
    for epoch in range(epochs):
        l_tr_g_temp = 0
        l_tr_d_temp = 0
        
        ############ Shuffle and Gen Fake Pairs ############
        shuffle(real_pairs)
        fake_pairs = []
        for i in range(n_train):
            nidx = randint(0,len(train_s1_emds)-1)
            while nidx == real_pairs[i][1]:
                nidx = randint(0,len(train_s1_emds)-1)
            fake_pairs.append((real_pairs[i][0],nidx))
        
        ############ Training #############
        start = 0
        end = 0
        while start < n_train:
            end = start + batch_size
            if end > n_train:
                end = n_train
            batch_size_ = end - start
            
            inp_img = [train_s1_imgs[x[0]] for x in real_pairs[start:end]]
            inp_text = [train_s1_emds[x[1]] for x in real_pairs[start:end]]
            inp_ftext = [train_s1_emds[x[1]] for x in fake_pairs[start:end]]
            
            real_img = Variable(torch.FloatTensor(inp_img).view(-1,img_shape[0],img_shape[1],img_shape[2])).type(torch.FloatTensor).cuda()
            real_text = Variable(torch.FloatTensor(inp_text).view(-1,desc_shape[0])).cuda()
            fake_text = Variable(torch.FloatTensor(inp_ftext).view(-1,desc_shape[0])).cuda()
            
            fake_img, mu, sigma = gen(real_text, batch_size=batch_size_)
            
            disc_real = disc(real_img, mu)
            disc_fake = disc(fake_img, mu)
            augm = cond_aug(embedding_dim, cond_dim)
            augm.cuda()
            mu_w, _, _ = augm(fake_text, batch_size=batch_size_)
            disc_wrong = disc(real_img, mu_w)
            
            optim_gen.zero_grad()
            optim_disc.zero_grad()
            
            ld_ = loss_disc(disc_real, disc_fake, disc_wrong, batch_size_)         
            ld_.backward(retain_graph=True)
            optim_disc.step()
            lg_ = loss_gen(disc_fake, mu, sigma, kl_lambda, batch_size_)
            lg_.backward(retain_graph=True)
            optim_gen.step()
            
            l_tr_g_temp += float(lg_.data)
            l_tr_d_temp += float(ld_.data)
            
            del inp_img, inp_text, inp_ftext, real_img, real_text, fake_text, disc_real, disc_fake, augm, mu_w, disc_wrong, ld_, lg_, batch_size_
            
            start += batch_size
            
        l_tr_g.append(l_tr_g_temp/n_train)
        l_tr_d.append(l_tr_d_temp/n_train)
        iter_tr.append(epoch+1)
        l = open('./observations/Loss_S1_'+ts+'.txt','a+')
        l.write('Epoch ' + str(epoch) + ': Generator Loss = ' + str(l_tr_g_temp/n_train) + \
               '         Discriminator Loss = ' + str(l_tr_d_temp/n_train)  + '\n')
        print('Epoch ' + str(epoch) + ': Generator Loss = ' + str(l_tr_g_temp/n_train) + \
               '         Discriminator Loss = ' + str(l_tr_d_temp/n_train))
        l.close()
    
    return l_tr_g, l_tr_d
        

In [ ]:
def train_s2(gen, disc, gen1, train_s2_imgs, train_s2_emds, batch_size=64, epochs=10, eta=0.0001, opt=optim.Adam,
             model_name='A_Model_Has_No_Name', kl_lambda=2.0, embedding_dim=1024, cond_dim=128):
    gen.cuda()
    disc.cuda()
    gen1.cuda()
    
    optim_gen = opt(gen.parameters(), lr=eta)
    optim_disc = opt(disc.parameters(), lr=eta)
    
    print('Training Stage 1...')
    
    l_tr_g = []
    l_tr_d = []
    iter_tr = []
    
#     n_train = len(train_s1_imgs)
    desc_shape = train_s2_emds[0].shape
    img_shape = train_s2_imgs[0].shape
    
    file_path = "./observations/"
    directory = os.path.dirname(file_path)
    if not os.path.exists(directory):
        os.makedirs(directory)
    file_path = "./saved_models/"
    directory = os.path.dirname(file_path)
    if not os.path.exists(directory):
        os.makedirs(directory)
    ts = strftime("%Y-%m-%d__%Hh%Mm%Ss_S1_" + model_name, gmtime())
    
    real_pairs = []
    for i in range(len(train_s2_imgs)):
        for j in range(n_desc_per_img):
            real_pairs.append((i,i * n_desc_per_img + j))
    
    n_train = len(real_pairs)
    print('Training Size = ',n_train)
    
    for epoch in range(epochs):
        l_tr_g_temp = 0
        l_tr_d_temp = 0
        
        ############ Shuffle and Gen Fake Pairs ############
        shuffle(real_pairs)
        fake_pairs = []
        for i in range(n_train):
            nidx = randint(0,len(train_s2_emds)-1)
            while nidx == real_pairs[i][1]:
                nidx = randint(0,len(train_s2_emds)-1)
            fake_pairs.append((real_pairs[i][0],nidx))
        
        ############ Training #############
        start = 0
        end = 0
        while start < n_train:
            end = start + batch_size
            if end > n_train:
                end = n_train
            batch_size_ = end - start
            
            inp_img = [train_s2_imgs[x[0]] for x in real_pairs[start:end]]
            inp_text = [train_s2_emds[x[1]] for x in real_pairs[start:end]]
            inp_ftext = [train_s2_emds[x[1]] for x in fake_pairs[start:end]]
            
            real_img = Variable(torch.FloatTensor(inp_img).view(-1,img_shape[0],img_shape[1],img_shape[2])).type(torch.FloatTensor).cuda()
            real_text = Variable(torch.FloatTensor(inp_text).view(-1,desc_shape[0])).cuda()
            fake_text = Variable(torch.FloatTensor(inp_ftext).view(-1,desc_shape[0])).cuda()
            
            fake_s1 = gen1(real_text, batch_size=batch_size_)[0]
            fake_img, mu, sigma = gen(real_text, fake_s1, batch_size=batch_size_)
            
            disc_real = disc(real_img, mu)
            disc_fake = disc(fake_img, mu)
            augm = cond_aug(embedding_dim, cond_dim)
            augm.cuda()
            mu_w, _, _ = augm(fake_text, batch_size=batch_size_)
            disc_wrong = disc(real_img, mu_w)
            
            optim_gen.zero_grad()
            optim_disc.zero_grad()
            
            ld_ = loss_disc(disc_real, disc_fake, disc_wrong, batch_size_)         
            ld_.backward(retain_graph=True)
            optim_disc.step()
            lg_ = loss_gen(disc_fake, mu, sigma, kl_lambda, batch_size_)
            lg_.backward(retain_graph=True)
            optim_gen.step()
            
            l_tr_g_temp += float(lg_.data)
            l_tr_d_temp += float(ld_.data)
            
            del inp_img, inp_text, inp_ftext, real_img, real_text, fake_text, disc_real, disc_fake, augm, mu_w, disc_wrong, ld_, lg_, batch_size_, fake_s1
            
            print(start)
            start += batch_size
            
        l_tr_g.append(l_tr_g_temp/n_train)
        l_tr_d.append(l_tr_d_temp/n_train)
        iter_tr.append(epoch+1)
        l = open('./observations/Loss_S1_'+ts+'.txt','a+')
        l.write('Epoch ' + str(epoch) + ': Generator Loss = ' + str(l_tr_g_temp/n_train) + \
               '         Discriminator Loss = ' + str(l_tr_d_temp/n_train)  + '\n')
        print('Epoch ' + str(epoch) + ': Generator Loss = ' + str(l_tr_g_temp/n_train) + \
               '         Discriminator Loss = ' + str(l_tr_d_temp/n_train))
        l.close()
    
    return l_tr_g, l_tr_d
        

In [ ]:
def test_s1(gen, disc, test_s1_data, test_text, model_name='A_Model_Has_No_Name'):
    print('Testing Stage 1...')
    
    n_test = len(test_s1_data)
    desc_shape = test_s1_data[0][1].shape
    
    ts = strftime("%Y-%m-%d__%Hh%Mm%Ss_S1_" + model_name, gmtime())
    file_path = "./gen_imgs/"
    directory = os.path.dirname(file_path)
    if not os.path.exists(directory):
        os.makedirs(directory)
    
    for i in range(n_test):
        text = Variable(torch.from_numpy(test_s1_data[i][1]).view(-1,desc_shape[0])).cuda()
        fake_img = gen(text)[0]
        out_shape = fake_img.shape
        fake_img_ = np.transpose(fake_img.cpu().view(out_shape[1],out_shape[2],out_shape[3]).data.numpy(),(1,2,0))
        idx = i // 2
        im = Image.fromarray(fake_img_.astype('uint8'))
        save_text = test_text[idx]
        if save_text[-1] == '.':
            save_text = save_text[:-1]
        im.save(file_path+ts+save_text+'.jpg')
        i += 1
        
        del text, fake_img, fake_img_, im

In [ ]:
def test_s2(gen, disc, gen1, test_s2_data, test_text, model_name='A_Model_Has_No_Name'):
    print('Testing Stage 2...')
    
    n_test = len(test_s2_data)
    desc_shape = test_s2_data[0][1].shape
    
    ts = strftime("%Y-%m-%d__%Hh%Mm%Ss_S2_" + model_name, gmtime())
    file_path = "./gen_imgs/"
    directory = os.path.dirname(file_path)
    if not os.path.exists(directory):
        os.makedirs(directory)
    
    for i in range(n_test):
        text = Variable(torch.from_numpy(test_s2_data[i][1]).view(-1,desc_shape[0])).cuda()
        fake_s1 = gen1(text)[0]
        fake_img = gen(text, fake_s1)[0]
        out_shape = fake_img.shape
        fake_img_ = np.transpose(fake_img.cpu().view(out_shape[1],out_shape[2],out_shape[3]).data.numpy(),(1,2,0))
        idx = i // 2
        im = Image.fromarray(fake_img_.astype('uint8'))
        save_text = test_text[idx]
        if save_text[-1] == '.':
            save_text = save_text[:-1]
        im.save(file_path+ts+save_text+'.jpg')
        i += 1
        
        del text, fake_s1, fake_img, fake_img_, im

In [ ]:
def driver(train_s1_data, test_s1_data, train_s2_data, test_s2_data, train_text, test_text, batch_size=64, 
           epochs_s1=10, epochs_s2=10, eta_s1=0.0001, eta_s2=0.0001, opt=optim.Adam, embedding_dim=1024, 
           cond_dim=128, noise_dim=100, ups_input_dim=1024, down_dim=128, num_residuals=4, 
           kl_lambda=2.0, model_name = 'A_Model_Has_No_Name'):
    print('######### Processing Model ',model_name,' ###############')
    
    print('Models Initializaed ==>')
    gen_s1 = stage1_gen(embedding_dim=embedding_dim, cond_dim=cond_dim, noise_dim=noise_dim, ups_input_dim=ups_input_dim, batch_size=batch_size)
    disc_s1 = stage1_disc(cond_dim=cond_dim, down_dim=down_dim, batch_size=batch_size)
    gen_s2 = stage2_gen(down_dim=down_dim, embedding_dim=embedding_dim, cond_dim=cond_dim, num_residuals=num_residuals, batch_size=batch_size)
    disc_s2 = stage2_disc(cond_dim=cond_dim, down_dim=down_dim, batch_size=batch_size)
    
    print('Training Begins ==>')
    l_tr_g_s1, l_tr_d_s1 = train_s1(gen_s1, disc_s1, train_s1_data, batch_size=batch_size, epochs=epochs_s1,
            eta=eta_s1, opt=optim.Adam, model_name=model_name, kl_lambda=kl_lambda,
            embedding_dim=embedding_dim, cond_dim=cond_dim)
    l_tr_g_s2, l_tr_d_s2 = train_s2(gen_s2, disc_s2, gen_s1, train_s2_data, epochs=epochs_s2, eta=eta_s2, opt=optim.Adam, model_name=model_name,
             kl_lambda=kl_lambda, embedding_dim=embedding_dim, cond_dim=cond_dim)
    
    print('Testing Begins ==>')
    test_s1(gen_s1, disc_s1, test_s1_data, test_text, model_name=model_name)
    test_s2(gen_s2, disc_s2, gen_s1, test_s2_data, test_text, model_name=model_name)
    
    print('Done! Check Out the results...')

In [ ]:
print('Data Input ==>')
train_s1_imgs, train_s1_emds, test_s1_imgs, test_s1_emds, train_s2_imgs, train_s2_emds, test_s2_imgs, test_s2_emds, train_text, test_text = read_input()
print('Train_S1_Data : ',len(train_s1_imgs))
print('Test_S1_Data : ',len(test_s1_imgs))
print('Train_S2_Data : ',len(train_s2_imgs))
print('Test_S2_Data : ',len(test_s2_imgs))
print('Train_S1_Data : ',len(train_s1_emds))
print('Test_S1_Data : ',len(test_s1_emds))
print('Train_S2_Data : ',len(train_s2_emds))
print('Test_S2_Data : ',len(test_s2_emds))
print('Train_Text : ',len(train_text))
print('Test_Text : ',len(test_text))

In [ ]:
batch_size = 64
epochs_s1 = 600
epochs_s2 = 600
eta_s1 = 0.0001
eta_s2 = 0.0001
opt = optim.Adam
embedding_dim = 1024
cond_dim = 128
noise_dim = 100
ups_input_dim = 1024
down_dim = 128
num_residuals = 4
kl_lambda = 2.0
model_name = 'Trial_1'

# driver(train_s1_data, test_s1_data, train_s2_data, test_s2_data, train_text, test_text, epochs_s1=epochs_s1, 
#        epochs_s2=epochs_s2, eta_s1=eta_s1, eta_s2=eta_s2, opt=opt, embedding_dim=embedding_dim, cond_dim=cond_dim,
#        noise_dim=noise_dim, ups_input_dim=ups_input_dim, down_dim=down_dim, num_residuals=num_residuals,
#         kl_lambda=kl_lambda, model_name = model_name)

In [ ]:
gen_s1 = stage1_gen(embedding_dim=embedding_dim, cond_dim=cond_dim, noise_dim=noise_dim, ups_input_dim=ups_input_dim)
disc_s1 = stage1_disc(cond_dim=cond_dim, down_dim=down_dim)
gen_s2 = stage2_gen(down_dim=down_dim, embedding_dim=embedding_dim, cond_dim=cond_dim, num_residuals=num_residuals)
disc_s2 = stage2_disc(cond_dim=cond_dim, down_dim=down_dim)

In [ ]:
batch_size = 4
# l_tr_g_s1, l_tr_d_s1 = train_s1(gen_s1, disc_s1, train_s1_imgs, train_s1_emds, batch_size=batch_size,
#             epochs=epochs_s1, eta=eta_s1, opt=opt, model_name=model_name, kl_lambda=kl_lambda,
#             embedding_dim=embedding_dim, cond_dim=cond_dim)
l_tr_g_s2, l_tr_d_s2 = train_s2(gen_s2, disc_s2, gen_s1, train_s2_imgs, train_s2_emds, batch_size=batch_size,
            epochs=epochs_s2, eta=eta_s2, opt=opt, model_name=model_name, kl_lambda=kl_lambda,
            embedding_dim=embedding_dim, cond_dim=cond_dim)

In [ ]:
x = [1,2,3,4,5,6,7,8,9]
shuffle(x)

In [ ]:
x